In [1]:
import numpy as np
import keras
import pickle as pk
import matplotlib.pyplot as plt
import os
from keras.applications.resnet50 import ResNet50
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input
from keras.layers import UpSampling2D, Input
from keras.models import Model

/home/paperspace/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# Define model
inp = Input(shape=(32, 32, 3), name='image_input')
x = UpSampling2D(size =(7,7))(inp)
resnet = ResNet50(weights='imagenet', include_top=True)

resnet.layers.pop()
resnet.outputs = [resnet.layers[-1].output]
resnet.layers[-1].outbound_nodes = []
resnet.summary()
output = resnet(x)

model = Model(inputs=inp, outputs=output)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
image_input (InputLayer)     (None, 32, 32, 3)         0         
_________________________________________________________________
up_sampling2d_1 (UpSampling2 (None, 224, 224, 3)       0         
_________________________________________________________________
resnet50 (Model)             multiple                  23587712  
Total params: 23,587,712
Trainable params: 23,534,592
Non-trainable params: 53,120
_________________________________________________________________


In [94]:
# Function to load a batch into memory
def load_batch(data_dir, batch_id):
    with open(os.path.join(data_dir, 'data_batch_%i' % batch_id), mode='rb') as file:
        batch = pk.load(file, encoding='latin1')
    feats = batch['data'].reshape((len(batch['data']), 3, 32, 32)).transpose(0, 2, 3, 1)
    lbls = batch['labels']
    return feats, lbls

# Load the first batch
feats, labels = load_batch( 'cifar10', 1)
for i in range(2,6):
    feats_temp, labels_temp = load_batch( 'cifar10', i)
    feats = np.append(feats,feats_temp, axis=0)
    labels = np.append(labels,labels_temp, axis=0)
label_names = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

### <center> Calculate Embeddings </center>

In [114]:
sample_size = 50000

In [121]:
import time
import json
comp_embeddings = True
dump_embeddings = True
if comp_embeddings:
    start = time.time()
    embeddings = model.predict(feats[:sample_size,:,:,:])
    print(time.time()-start)
    
if dump_embeddings:
    with open(f'embeddings/embeddingsresnet50_{sample_size}.json', 'w') as outfile:
        json.dump([embeddings.tolist(),labels[:sample_size].tolist()], outfile)


269.3301594257355


### <center>Fine tune last layer </center>

In [97]:
def split_train_dev_test(feats, labels):
    n = len(labels)
    indices = np.random.permutation(n)
    training_idx, dev_idx,test_idx = indices[:int(0.9*n)], indices[int(0.9*n):int(0.95*n)], indices[int(0.95*n):]
    feats_train,feats_dev, feats_test = feats[training_idx,:],feats[dev_idx,:], feats[test_idx,:]
    labels_train,labels_dev, labels_test = np.array([labels[i] for i in training_idx]),np.array([labels[i] for i in dev_idx]), np.array([labels[i] for i in test_idx])
    return [feats_train,feats_dev, feats_test],[labels_train,labels_dev, labels_test]

In [98]:
from keras.utils.np_utils import to_categorical
n_feats = (feats - np.mean(feats))/np.std(feats)
c_labels = to_categorical(labels, 10)
[feats_train,feats_dev, feats_test],[labels_train,labels_dev, labels_test] = split_train_dev_test(n_feats, c_labels)

In [54]:
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers.normalization import BatchNormalization

In [91]:
# Define model
inp = Input(shape=(32, 32, 3), name='image_input')
x = UpSampling2D(size =(7,7))(inp)
resnet = ResNet50(weights='imagenet', include_top=True)

resnet.layers.pop()
resnet.outputs = [resnet.layers[-1].output]
resnet.layers[-1].outbound_nodes = []
x = resnet(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
output = Dense(10, activation='softmax')(x)
model = Model(inputs=inp, outputs=output)
for layer in model.layers[:-1]:
    layer.trainable = False

In [62]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
image_input (InputLayer)     (None, 32, 32, 3)         0         
_________________________________________________________________
up_sampling2d_10 (UpSampling (None, 224, 224, 3)       0         
_________________________________________________________________
resnet50 (Model)             multiple                  23587712  
_________________________________________________________________
batch_normalization_2 (Batch (None, 2048)              8192      
_________________________________________________________________
dropout_3 (Dropout)          (None, 2048)              0         
_________________________________________________________________
dense_7 (Dense)              (None, 10)                20490     
Total params: 23,616,394
Trainable params: 20,490
Non-trainable params: 23,595,904
___________________________________________________________

In [99]:
model.compile(loss='categorical_crossentropy',optimizer='adam', metrics=['accuracy'])
model.fit(feats_train, labels_train, epochs=2, batch_size=128)

Epoch 1/2
45000/45000 [==============================] - 184s 4ms/step - loss: 0.9250 - acc: 0.7062
Epoch 2/2
45000/45000 [==============================] - 164s 4ms/step - loss: 0.7876 - acc: 0.7410


In [109]:
model.evaluate(feats_dev,labels_dev)

ValueError: Error when checking input: expected image_input to have 4 dimensions, but got array with shape (2500, 32, 3)

Dense(3) -> 0.1 on dev
Dense(3) + Dropout 0.5 + 5 epochs + 20K -> 0.101 on dev 
Dense(3) + Dropout 0.5 + 5 epochs + 40K -> 0.0855 on dev 
Dense(3) + Dropout 0.25 + 5 epochs + 40K -> 0.0855 on dev 
Dense(3) + Dropout 0.5 + 2 epochs + 50k -> 0.09

In [46]:
from collections import Counter

In [101]:
labels_predicted = model.predict(feats_dev)

In [110]:
np.sum(np.argmax(labels_predicted, axis=1)==np.argmax(labels_dev,axis=1))/len(labels_dev)

0.094